<a href="https://colab.research.google.com/github/faizanali2005/googlecolab/blob/main/lab08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Complete Colab Notebook: Minimax, Alpha-Beta, and Tic-Tac-Toe

!pip install pandas
import pandas as pd

# ==========================================
# Tic-Tac-Toe Board & Players
# ==========================================
origBoard = ["O",1,"X","X",4,"X",6,"O","O"]
huPlayer = "O"
aiPlayer = "X"

# ==========================================
# Helper Functions
# ==========================================
def emptyIndexies(board):
    return [i for i, x in enumerate(board) if x != "O" and x != "X"]

def winning(board, player):
    return (
        (board[0]==player and board[1]==player and board[2]==player) or
        (board[3]==player and board[4]==player and board[5]==player) or
        (board[6]==player and board[7]==player and board[8]==player) or
        (board[0]==player and board[3]==player and board[6]==player) or
        (board[1]==player and board[4]==player and board[7]==player) or
        (board[2]==player and board[5]==player and board[8]==player) or
        (board[0]==player and board[4]==player and board[8]==player) or
        (board[2]==player and board[4]==player and board[6]==player)
    )

# ==========================================
# Minimax Algorithm
# ==========================================
def minimax(newBoard, player):
    availSpots = emptyIndexies(newBoard)

    if winning(newBoard, huPlayer):
        return {"score": -10}
    elif winning(newBoard, aiPlayer):
        return {"score": 10}
    elif len(availSpots) == 0:
        return {"score": 0}

    moves = []
    for i in availSpots:
        move = {}
        move["index"] = i
        newBoard[i] = player
        if player == aiPlayer:
            result = minimax(newBoard, huPlayer)
            move["score"] = result["score"]
        else:
            result = minimax(newBoard, aiPlayer)
            move["score"] = result["score"]
        newBoard[i] = move["index"]
        moves.append(move)

    bestMove = None
    if player == aiPlayer:
        bestScore = -10000
        for i, m in enumerate(moves):
            if m["score"] > bestScore:
                bestScore = m["score"]
                bestMove = i
    else:
        bestScore = 10000
        for i, m in enumerate(moves):
            if m["score"] < bestScore:
                bestScore = m["score"]
                bestMove = i

    return moves[bestMove]

# ==========================================
# Example: Best Move
# ==========================================
best_move = minimax(origBoard, aiPlayer)
print("Best move for AI:", best_move)

# ==========================================
# Alpha-Beta Pruning Trace Example
# ==========================================
node_children = {
    "MAX": ["MIN1","MIN2","MIN3"],
    "MIN1": ["3","12"],
    "MIN2": ["8","2"],
    "MIN3": ["4","6"]
}
leaf_values = {"3":3,"12":12,"8":8,"2":2,"4":4,"6":6}
trace = []
step_counter = 1

def alphabeta_trace(node, isMax, alpha=-10000, beta=10000):
    global step_counter
    if node in leaf_values:
        trace.append({"Step":step_counter,"Node":node,"Node Type":"Leaf",
                      "Alpha":alpha,"Beta":beta,"Action":f"Return {leaf_values[node]}"})
        step_counter += 1
        return leaf_values[node]

    node_type = "MAX" if isMax else "MIN"
    value = -999 if isMax else 999
    trace.append({"Step":step_counter,"Node":node,"Node Type":node_type,
                  "Alpha":alpha,"Beta":beta,"Action":"Start evaluating children"})
    step_counter += 1

    for child in node_children[node]:
        child_value = alphabeta_trace(child, not isMax, alpha, beta)
        if isMax:
            value = max(value, child_value)
            alpha = max(alpha, value)
        else:
            value = min(value, child_value)
            beta = min(beta, value)

        action = f"Update value={value}, α={alpha}, β={beta} after visiting {child}"
        trace.append({"Step":step_counter,"Node":node,"Node Type":node_type,
                      "Alpha":alpha,"Beta":beta,"Action":action})
        step_counter += 1

        if beta <= alpha:
            trace.append({"Step":step_counter,"Node":node,"Node Type":node_type,
                          "Alpha":alpha,"Beta":beta,"Action":f"Prune remaining children of {node}"})
            step_counter += 1
            break
    return value

root_value = alphabeta_trace("MAX", True)
print("Alpha-Beta optimal value at root:", root_value)
df_trace = pd.DataFrame(trace)
df_trace

# ==========================================
# Activity 6: Node Expansion Comparison
# ==========================================
b = 3  # branching factor
d = 4  # depth

# Minimax nodes
minimax_nodes = sum([b**i for i in range(d+1)])
# Alpha-Beta nodes best case (approx)
alphabeta_nodes_best = sum([b**i for i in range(d//2+1)])
# Alpha-Beta nodes worst case = minimax
alphabeta_nodes_worst = minimax_nodes

print("\nNode Expansion Comparison")
print(f"Minimax total nodes: {minimax_nodes}")
print(f"Alpha-Beta best case nodes (approx): {alphabeta_nodes_best}")
print(f"Alpha-Beta worst case nodes: {alphabeta_nodes_worst}")
print("Alpha-Beta pruning is faster than Minimax in best case because it prunes branches that cannot affect final decision.")

# ==========================================
# Activity 5: α and β Table Example
# ==========================================
print("\nAlpha-Beta Step-by-Step Table")
df_trace


Best move for AI: {'index': 4, 'score': 10}
Alpha-Beta optimal value at root: 4

Node Expansion Comparison
Minimax total nodes: 121
Alpha-Beta best case nodes (approx): 13
Alpha-Beta worst case nodes: 121
Alpha-Beta pruning is faster than Minimax in best case because it prunes branches that cannot affect final decision.

Alpha-Beta Step-by-Step Table


,Step,Node,Node Type,Alpha,Beta,Action
0,1,MAX,MAX,-10000,10000,Start evaluating children
1,2,MIN1,MIN,-10000,10000,Start evaluating children
2,3,3,Leaf,-10000,10000,Return 3
3,4,MIN1,MIN,-10000,3,"Update value=3, α=-10000, β=3 after visiting 3"
4,5,12,Leaf,-10000,3,Return 12
5,6,MIN1,MIN,-10000,3,"Update value=3, α=-10000, β=3 after visiting 12"
6,7,MAX,MAX,3,10000,"Update value=3, α=3, β=10000 after visiting MIN1"
7,8,MIN2,MIN,3,10000,Start evaluating children
8,9,8,Leaf,3,10000,Return 8
9,10,MIN2,MIN,3,8,"Update value=8, α=3, β=8 after visiting 8"
